# Overview

## Properties of remote computing

Resources such as compute and storage can be accessed anywhere, virtually unlimited, data is permanent (resilient against crashes), easily shareable, many different types of compute resources

## Characteristics of Cloud based Computing

On-demand resources (pay as you go)   
Resource Scalability (increased resources allocated to you as you need more)  
API-based access

# Virtual Machines

Virtual Machine is a process which is an OS running on existing system's OS.  
The existing system's OS is referred to as the **hypervisor**. 
VMs are useful because:
1. Easy way to use different OS
2. Fully utilize node capacity
3. Isolation- run shitty code on separate VMs

Key aspects are Host (physical machine), guest (VM on some OS), VMM/Hypervisor (software layer performing virtualization of the host)

## Virtualization

Suite of techniques that enable multiple Guests each running a separate OS to be run on the same physical machine.   
Challenges: 
1. Isolate guests from each other
2. Support different OS
3. Keep performance overhead small  
  
Full Virtualization - exact copy of the host hardware exposed to guest OS - no change required to guest OS, hard for x86    
Paravirtualization - similar to full virtualization, but not the exect copy of the host hardware is exposed, requires modification to Guest OS but no modification to guest applications

## XEN

Hypervisor that uses paravirtualization technique, requires modification of guest OS.   
XEN runs as the most priveleged software (ring 0), whereas guest OS runs at a lower privelege level (ring 1)

### XEN and syscalls

Naive setup: Guest OS calls XEN to execute each exception, XEN calls processor to execute syscall, processor executes syscall and returns control to XEN which returns control to OS.  
**This is inefficient as fuck**  
Faster exception handlers: Guest OS calls XEN to register an exception handler table. XEN validates the table, and places it in memory where the processor can find it. The guest OS executes a system call, and the processor directly accesses the system call handler table without XEN, and returns back to XEN after completion.   
**The idea here is we removed a level of abstraction, XEN, by storing exception handler table into memory, which made syscalls/exceptions much faster**

### XEN and Page Faults

What is a page fault? When the CPU tries to access a page not mapped in the virtual address space of a process 
Normal demand paging:  
Page table used to translate virtual addresses to physical addresses. Each process needs its own table, and the OS setsup the page table for a process when it starts the process.

### Normal Page fault handling

1. MMU notices that a particular virtual address cannot be translated to a physical address, and then notifies the CPU about this.
2. CPU sends an interrupt to the OS called the page fault
3. OS page fault handler is at a fixed memory location, and control transferred to this page fault handler.  
4. OS uses a page replacement algorithm to determine whether or not a page will be replaced and written to disk or not.
5. It reads the new code into that newly emptied page frame, and updates the page table entry. It then re-executes the faulting instruction, which should work.  
  
CPU works with virtual addresses, MMU contains page table per process which is used to translate virtual addresses to physical addresses, and this page table is set up by the OS. When a page fault occurs, the OS helps resolve it.

### XEN page fault handling

1. Guest OS registers page table with XEN
2. Processor encounters page fault, and transfers control to XEN (hardware exception table set to do this at boot time)
3. XEN reads the faulting page address from the processor register and saves it in the execution stack frame of the guest OS, and resolves it
4. XEN restarts the faulting instruction from the execution stack frame of the guest OS   
  
CPU works with virtual addresses, Guest OS registers per process page table with XEN, XEN sets up the page table inside MMU when it schedules a guest OS. MMU contains page table for a process executing within the Guest OS. At boot time, XEN sets the page fault handler to transfer control to itself. WHen the CPU encounters a page fault, control is transferred to XEN and XEN resolves the page fault

### Virtualizing I/O

XEN manages I/O devices, and the guest OS calls XEN for I/O.   
XEN uses shared memory regions between itself and the guest OS as a fast data transfer mechanism, and there's an asynchronous event mechanism much like DMA to notify the guest OS when I/O is complete

### Terms and Concepts

Domains: Guest OS/VMs are called Domains.  
Domain0: priveleged comain created by XEN on startup, control center for creating and managing other domains

### VMs in the Cloud

Must support different hardwars and supervisors, and support the provisioning of Virtual Machines remotely (i.e. allow API access to spin up VMs)

### Libvirt

Node/Host: Physical Machine   
Hypervisor: Layer of software that virtualizes a node in a set of virtual machines    
Domain: An instance of an OS running on a Node   
Goal: provide a common and stable layer sufficient to securely manage domains on a node.   
  
Libvirt daemon (background process - remember you can run things in the background with &)   
All interactions go through this daemon, and local communication happens over Unix sockets. Many available methods, and various Domains (Guest OS) defined using XML

# EC2

Instance Type- determines the hardware of the host computer used for launching your instances.  
Instances- Running VM.    
AMI- OS definition, multiple instatnces can be launched from the same AMI  
Goal of instance types is to support different kinds of applications.   
Perks: General purpose, compute and memory optimized, accelerated computing, storage optimized

### AMI - Amazon Machine Image

OS image that should be used for booting VM.  
Information included is hardware architecture on which AMI is built, virtualization type for which the AMI is built, where to find root volume, which contains instructions on how to boot the AMI

### EC2 run_instances

1. Find out a hardware node corresponding to the provided instance type and has enough capacity
2. Fetch specified AMI
3. Create volumes on the Elastic Block Storage corresponding to the block device mapping
4. Invoke the libvirt driver on the selected Node to start a domain using the AMI fetched

### EC2 Virtualization

Paravirtual, and HVM (Hardware virtual machine)- uses special virtualization extensions, allows running guest OS without any modifications

### EC2 Instance initialization

Initializing an instance at startup contains user data and is done via cloud-init. User data is used when initialization is done using bash scripts. Logs stored in /var/log/cloud-init-output.log.   
Various use cases including an instance with a specific applicatino running, enabling remote logins, injecting app specific config files

### Security Groups

Security groups act as a virtual firewall, you can choose to allow traffic on specific ports. When you launch an instqance, you can associate one or more security groups with an instance. Rules need to hbe added to the security groups that control traffic to/from the instance.   
By default, all outbound traffic is allowed, you can allow certain protocols, port ranges, source/destinations, and all traffic (0.0.0.0/0)   
Each account gets a default security group per VPC (Virtual private cloud, abstraction for managing set of AWS resources as a unit. During instance creation is is inside the default VPC).

### CIDR

IP Address: 32 bit, divided into Network number and host number.   
CIDR Addressing: 
IP and Mask, mask is used to obtain network number. Indicates how many most significant bits should be treated as 1 while filtering.  
Written as IP Address/Mask: 1272.16.0.0/16: Mask is 16 bits in size, 16 most significant bits should be treated as 1.   
Mask is used in routing table to determine how to route a packet, used in security group rules to detemine wheter to accept a packet

### SSH Key Pair

Use an SSH key pair to connect to the EC2 instance, Public key stored by Amazon, private key stored by you.  
When the user logs in:
1. EC2 instance is authenticated by verifying the host key
2. User SSH client creates a unique message and encrypts it with the User's private key
3. EC2 instance uses User's public key to verify the message by decrypting it

# EC2